In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 

In [2]:
df = pd.read_csv("sample-data.csv")
df.head()

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."


Создаём функцию для выдачи названия товара по id:

In [3]:
def item(item_id):
    return df.loc[df['id'] == item_id]['description'].tolist()[0].split(' - ')[0]

Создаём функцию для нахождения наиболее похожих товаров по матрице расстояний:

In [7]:
def recommend(sim_matrix, item_id, num):
    index=df.index[df['id']==item_id].tolist()[0] #Определяем номер товара в матрице расстояний
    similar_indices = sim_matrix[index].argsort()[:-(num+2):-1]# Находим номера похожих товаров
    print("Топ " + str(num) + " товаров, похожих на " + item(item_id) + ":")
    print("-------")
    for ind in similar_indices[1:]:
        print("Товар " + item(df['id'][ind]) + " (score:" + str(sim_matrix[index][ind]) + ")")

TF-IDF

In [8]:
#Векторизуем весь датасет и считаем косинусное расстояние с помощью linear_kernel
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['description'])

cosine_similarities_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
recommend(cosine_similarities_tfidf, item_id=56, num=5)

Топ 5 товаров, похожих на Iceberg t-shirt:
-------
Товар Rockpile t-shirt (score:0.4376322029029032)
Товар Wind path t-shirt (score:0.37187692279165374)
Товар Text logo t-shirt (score:0.37141815057999866)
Товар Squid t-shirt (score:0.3516576070144953)
Товар Peregrine t-shirt (score:0.3474259633698038)


WORD2VEC

In [13]:
pip install --upgrade gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
Note: you may need to restart the kernel to use updated packages.


In [14]:
import gensim
import gensim.downloader
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ekaterina\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Создадим функцию для токенизации текста:

In [15]:
def tokenize_text(text):
    tokenizer = RegexpTokenizer(r"[\w-]+")# Выделим отдельные слова, в том числе с дефисом, с помощью регулярных выражений
    html_pattern = re.compile('<.*?>')# удалим html разметку
    clean_text = tokenizer.tokenize(html_pattern.sub('', text.lower()))
    stops = set(stopwords.words("english"))# удалим все стоп слова
    sentence = [w for w in clean_text if not w in stops]
    return sentence

Создадим функцию для заполнения марицы векторов по корпусу текстов.
В качестве вектора для одного текста будем использовать среднее значение word2vec отдельных слов.

In [17]:
def create_embed_matrix(texts):
    embed_matrix=np.zeros((len(texts),glove_vectors.vector_size))
    for ind, text in enumerate(texts):
        mean_word2vec=np.zeros(glove_vectors.vector_size)
        num_words=0
        tokenized_text=tokenize_text(text)
        #Перебор всех слов из текста и суммирование векторов
        for word in tokenized_text:
            if word in glove_vectors.index_to_key:
                mean_word2vec+=glove_vectors[word]
                num_words+=1
        embed_matrix[ind]=mean_word2vec/num_words
        #Нормализация финального вектора для одного тектса
        embed_matrix[ind]=embed_matrix[ind]/np.sqrt(np.sum(embed_matrix[ind]**2)) 
    return embed_matrix

In [18]:
#В качестве векторов используем embedding модели glove-wiki-gigaword-300
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [19]:
# Заполняем матрицу векторов и считаем попарное косинусное расстояние
mean_word2vec_matrix=create_embed_matrix(df['description'].values)

In [20]:
cosine_similarities_word2vec = linear_kernel(mean_word2vec_matrix, mean_word2vec_matrix)

In [21]:
recommend(cosine_similarities_word2vec, item_id=56, num=5)

Топ 5 товаров, похожих на Iceberg t-shirt:
-------
Товар Squid t-shirt (score:0.9627716069870447)
Товар Flying fish 2 t-shirt (score:0.9627118179373375)
Товар Tarpon t-shirt (score:0.9568614282247329)
Товар City by the sea t-shirt (score:0.9542965417873863)
Товар Wind path t-shirt (score:0.9536632205107418)
